# 7: Feedback - Human-in-the-Loop and Approval Workflows for Azure OpenAI Agents

This notebook demonstrates how to implement feedback and approval workflows in Azure OpenAI agents, including:
- Human-in-the-loop approval for high-risk or sensitive content
- Risk assessment using Azure OpenAI
- Approval request and decision schemas
- Batch approval and workflow management

These patterns are essential for responsible AI, compliance, and production-grade deployments.

In [1]:
# Install required packages
# !pip install openai azure-identity python-dotenv pydantic

In [2]:
import os
import json
from typing import Optional, Dict, List, Callable
from datetime import datetime
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

True

## Azure OpenAI Client Initialization
This function initializes the Azure OpenAI client using either API key or Managed Identity.

In [3]:
def get_azure_openai_client():
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
    if not endpoint:
        raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is required")
    if api_key:
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
    else:
        credential = DefaultAzureCredential()
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=credential,
            api_version=api_version,
        )
    return client

## Approval Request and Decision Schemas
We use Pydantic models to define the structure for approval requests and decisions.

In [4]:
class ApprovalRequest(BaseModel):
    id: str = Field(description="Unique identifier for the request")
    content: str = Field(description="Content to be reviewed")
    type: str = Field(description="Type of content (email, document, response, etc.)")
    risk_level: str = Field(description="Risk level: low, medium, high, critical")
    requester: str = Field(description="Who requested the approval")
    timestamp: str = Field(description="When the request was created")
    context: Optional[str] = Field(None, description="Additional context for the approval")

class ApprovalDecision(BaseModel):
    approved: bool = Field(description="Whether the content was approved")
    feedback: Optional[str] = Field(None, description="Human feedback or comments")
    modifications: Optional[str] = Field(None, description="Suggested modifications")
    approver: str = Field(description="Who made the approval decision")
    timestamp: str = Field(description="When the decision was made")

## Risk Assessment with Azure OpenAI
This function uses Azure OpenAI to assess the risk level of content for public or customer communication.

In [5]:
def assess_content_risk(content: str) -> str:
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    risk_assessment_prompt = f"""
Assess the risk level of this content for public or customer communication:

Content: "{content}"

Consider:
- Potential for misunderstanding
- Legal or compliance issues
- Brand reputation impact
- Sensitive information disclosure
- Emotional or controversial topics

Return only one word: low, medium, high, or critical
"""
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[{"role": "user", "content": risk_assessment_prompt}],
            temperature=0.1,
            max_tokens=50,
        )
        risk_level = response.choices[0].message.content.strip().lower()
        if risk_level in ["low", "medium", "high", "critical"]:
            return risk_level
        else:
            return "medium"  # Default fallback
    except Exception as e:
        print(f"Risk assessment failed: {e}")
        return "high"  # Conservative default

## Human Approval Workflow
This function provides a simple human-in-the-loop approval interface for content.

In [7]:
def get_human_approval(content: str, content_type: str = "general") -> bool:
    print("=" * 60)
    print(f"APPROVAL REQUEST - {content_type.upper()}")
    print("=" * 60)
    print(f"Content to review:\n{content}\n")
    print("=" * 60)
    while True:
        response = input("Approve this content? (y)es / (n)o / (m)odify / (v)iew again: ").lower().strip()
        if response.startswith('y'):
            return True
        elif response.startswith('n'):
            return False
        elif response.startswith('m'):
            modification = input("What modifications would you suggest? ")
            print(f"Modifications noted: {modification}")
            return False
        elif response.startswith('v'):
            print(f"\nContent:\n{content}\n")
            continue
        else:
            print("Please enter 'y' for yes, 'n' for no, 'm' for modify, or 'v' to view again.")


## Content Generation with Human Feedback
This function generates content with Azure OpenAI and optionally requires human approval based on risk assessment.

In [8]:
def intelligence_with_human_feedback(prompt: str, require_approval: bool = True) -> dict:
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=1000,
        )
        draft_content = response.choices[0].message.content
        result = {"content": draft_content, "generated_at": datetime.now().isoformat(), "prompt": prompt}
        if require_approval:
            risk_level = assess_content_risk(draft_content)
            result["risk_level"] = risk_level
            needs_approval = risk_level in ["high", "critical"]
            if needs_approval:
                approved = get_human_approval(draft_content, "AI Generated Content")
                result["approved"] = approved
                result["approval_required"] = True
                result["status"] = "approved" if approved else "rejected"
            else:
                result["approved"] = True
                result["approval_required"] = False
                result["status"] = f"auto_approved (risk level: {risk_level})"
        else:
            result["approved"] = True
            result["approval_required"] = False
            result["status"] = "no_approval_needed"
        return result
    except Exception as e:
        return {"error": str(e), "status": "error", "approved": False}

## Approval Workflow Manager
This class manages approval requests, processes approvals, and tracks statistics.

In [10]:
class ApprovalWorkflow:
    def __init__(self):
        self.client = get_azure_openai_client()
        self.deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
        self.pending_approvals: List[ApprovalRequest] = []
        self.approval_history: List[Dict] = []
    def submit_for_approval(self, content: str, content_type: str = 'general', requester: str = 'system', context: Optional[str] = None) -> str:
        risk_level = assess_content_risk(content)
        request_id = f'req_{len(self.pending_approvals) + 1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}'
        approval_request = ApprovalRequest(
            id=request_id,
            content=content,
            type=content_type,
            risk_level=risk_level,
            requester=requester,
            timestamp=datetime.now().isoformat(),
            context=context
        )
        self.pending_approvals.append(approval_request)
        print(f'📋 Approval request {request_id} submitted (Risk: {risk_level})')
        return request_id
    def process_approval(self, request_id: str, approver: str = 'human') -> Optional[ApprovalDecision]:
        request = next((req for req in self.pending_approvals if req.id == request_id), None)
        if not request:
            print(f'❌ Approval request {request_id} not found')
            return None
        print(f'📋 Processing approval request: {request_id}')
        print(f'Type: {request.type}')
        print(f'Risk Level: {request.risk_level}')
        print(f'Requester: {request.requester}')
        if request.context:
            print(f'Context: {request.context}')
        print(f'Submitted: {request.timestamp}')
        approved = get_human_approval(request.content, request.type)
        feedback = None
        modifications = None
        if not approved:
            feedback = input('Please provide feedback (optional): ').strip()
            modifications = input('Suggest modifications (optional): ').strip()
        decision = ApprovalDecision(
            approved=approved,
            feedback=feedback if feedback else None,
            modifications=modifications if modifications else None,
            approver=approver,
            timestamp=datetime.now().isoformat()
        )
        self.approval_history.append({"request": request.model_dump(), "decision": decision.model_dump()})
        self.pending_approvals.remove(request)
        return decision
    def process_all_pending(self, approver: str = 'human') -> List[ApprovalDecision]:
        decisions = []
        while self.pending_approvals:
            request = self.pending_approvals[0]
            decision = self.process_approval(request.id, approver)
            if decision:
                decisions.append(decision)
        return decisions
    def get_pending_count(self) -> int:
        return len(self.pending_approvals)
    def get_approval_stats(self) -> Dict:
        if not self.approval_history:
            return {"total": 0, "approved": 0, "rejected": 0, "approval_rate": 0.0}
        total = len(self.approval_history)
        approved = sum(1 for item in self.approval_history if item["decision"]["approved"])
        rejected = total - approved
        approval_rate = approved / total if total > 0 else 0.0
        return {"total": total, "approved": approved, "rejected": rejected, "approval_rate": approval_rate}


## Example: Content Generation and Approval
Let's generate content and walk it through the approval workflow.

In [11]:
# Generate content and require approval
result = intelligence_with_human_feedback("Write a short poem about technology")
print(f'Result: {result}')

Result: {'content': 'A hum of circuits, silent light,  \nWires weaving through the night.  \nIdeas dance on glass and code,  \nWorlds connected, stories told.\n\nBright screens shimmer, casting dreams,  \nBoundless knowledge flows in streams.  \nYet in the glow, let’s not forget  \nThe human heart, our greatest tech.', 'generated_at': '2025-07-29T11:11:22.986711', 'prompt': 'Write a short poem about technology', 'risk_level': 'low', 'approved': True, 'approval_required': False, 'status': 'auto_approved (risk level: low)'}


## Example: Approval Workflow Manager
Let's submit multiple items for approval and process them.

In [12]:
workflow = ApprovalWorkflow()
req1 = workflow.submit_for_approval("Thank you for your purchase! Your order will arrive in 2-3 business days.", 'customer_email', 'customer_service')
req2 = workflow.submit_for_approval("We're sorry to inform you that your account has been temporarily suspended due to suspicious activity.", 'account_notification', 'security_team', 'Automated fraud detection triggered')
print(f'Pending approvals: {workflow.get_pending_count()}')
decision1 = workflow.process_approval(req1)
print(f'Decision: {decision1.model_dump_json(indent=2) if decision1 else 'None'}')
stats = workflow.get_approval_stats()
print(f'Approval stats: {stats}')

📋 Approval request req_1_20250729_111144 submitted (Risk: low)
📋 Approval request req_2_20250729_111145 submitted (Risk: medium)
Pending approvals: 2
📋 Processing approval request: req_1_20250729_111144
Type: customer_email
Risk Level: low
Requester: customer_service
Submitted: 2025-07-29T11:11:44.274928
APPROVAL REQUEST - CUSTOMER_EMAIL
Content to review:
Thank you for your purchase! Your order will arrive in 2-3 business days.

Decision: {
  "approved": true,
  "feedback": null,
  "modifications": null,
  "approver": "human",
  "timestamp": "2025-07-29T11:11:51.421310"
}
Approval stats: {'total': 1, 'approved': 1, 'rejected': 0, 'approval_rate': 1.0}


# Summary
In this notebook, we've implemented feedback and approval workflows for Azure OpenAI agents, including human-in-the-loop, risk assessment, and batch approval management.